# F-тест: Совместная значимость
Тестируется гипотеза
$$H_0:\beta_1=\cdots=\beta_J=0 $$
Тестовая статистика (неробастная) $$F=\frac{R^2-R^2_{restr}}{1-R^2}\cdot\frac{n-k-1}{J}=\frac{RSS_{restr}-RSS}{RSS}\cdot\frac{n-k-1}{J}$$
$R^2_{restr}, RSS_{restr}$ – из промежуточной регрессии с ограничениями

Критическое значение $$F_{cr}=F_{df1=J, df2=n-k-1}(\alpha)$$

Гипотеза отвергается если $F>F_{cr}$ или $P<\alpha$

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col # вывод подгонки
from scipy.stats import f # F-распределение

## Sleep equation 1
Для датасета `sleep75` рассмотрим регрессию **sleep на totwrk, age, male, south, smsa, yngkid, marr, union**

Результаты подгонки

In [ ]:
sleep_df = pd.read_csv('https://raw.githubusercontent.com/artamonoff/Econometrica/master/python-notebooks/data-csv/sleep75.csv')
mod1 = smf.ols(formula='sleep~totwrk+age+male+south+smsa+yngkid+marr+union', data=sleep_df).fit()
summary_col(mod1, stars=True)

Потестируем совместную значимость `smsa, yngkid, marr, union`, т.е. гипотезу
$$H_0:\beta_{smsa}=\beta_{yngkid}=\beta_{marr}=\beta_{union}=0$$
Критическое значений: df1=4, df2=n-k-1

In [ ]:
f.ppf(q=1-0.05, dfn=4, dfd=mod1.df_resid)

### Первый способ: через спецификацию гипотезы

In [ ]:
print( mod1.f_test('smsa=yngkid=marr=union=0') )

In [ ]:
print( mod1.wald_test('smsa=yngkid=marr=union=0', use_f=True) )

### Второй способ: через вспомогательную регрессию в ограничениями

In [ ]:
mod1_restr = smf.ols(formula='sleep~totwrk+age+male+south', data=sleep_df).fit()
mod1.compare_f_test(mod1_restr)

### Явное вычисление

In [ ]:
(mod1.rsquared-mod1_restr.rsquared)/(1-mod1.rsquared)*mod1.df_resid/4

In [ ]:
(mod1_restr.ssr-mod1.ssr)/mod1.ssr*mod1.df_resid/4